In [2]:
# TODO: Determine position size: Dollar Volatility = N × Dollars per Point. Unit = 1% of porfolio / Dollar Volatility
# TODO: Reduce trading size when account amount decreases (10% overall decrease -> 20% trading amount decrease), reinstate when back to initial starting amount
# TODO: Implement System 2
# TODO: Ignoring breakouts (System 1) as previous one was a winner (regardless of long or short), failsafe is System 2 entry (System 2 does not have this breakout filter)
# Implemented: System 1 Longs, pyramiding up to 5 additional orders (1/2 N increments)
# Implemented: fixed selling over multiple orders (observation: some work better when selling over a period of time vs all at once)
# Implemented: both longs and shorts can be executed (Need to run tests to verify it's working properly)

from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])

# Import the backtrader platform
import backtrader as bt

def closeAllPositions(self):
    for data in self.datas:
        size = self.getposition(data).size
        if  size != 0:
            self.order = self.close(data)

def longPosition(self):
    stop2N = self.previousOrder - 2 * self.atr[-1]
    pyramidHalfN = self.previousOrder + 1/2 * self.atr[-1]
    # in the market, see if we can pyramid our position
    if self.pyramid < 5 and self.dataopen[0] > pyramidHalfN:
        # BUY, BUY, BUY!!! (with default parameters)
        self.log('BUY CREATE PYRAMID, current open price: %.2f' % self.dataopen[0])
        self.log('N value: %.2f' % self.atr[-1])
        self.log('Pyramid: %.2f' % self.pyramid)
        self.log('N buy in price: %.2f' % pyramidHalfN)
        self.pyramid += 1
        # Keep track of the created order to avoid a 2nd order
        self.order = self.buy()
        self.log('New previousOrder, should be same as BUY EXECUTED: %.2f' % self.previousOrder)
        
    # Already in the market ... we might sell
    if self.s1ExitLong[-1] > self.dataopen[0]:
        # SELL, SELL, SELL!!! (with all possible default parameters)
        # Below system low exit price
        self.log('SELL CREATE, 10 day low: %.2f' % self.s1ExitLong[-1])
        self.log('exit low: %.2f' % self.dataopen[0])
        # Keep track of the created order to avoid a 2nd order
        self.pyramid = 0
        closeAllPositions(self)
                
    elif self.dataopen[0] < stop2N:
        # Below 2N stop
        stopPrice = stop2N
        self.log('___________SELL CREATE, N stop: %.2f' % stopPrice)
        self.log('Current Price: %.2f' % self.dataopen[0])
        self.log('N value: %.2f' % self.atr[-1])
        # Keep track of the created order to avoid a 2nd order
        self.pyramid = 0
        closeAllPositions(self)

def shortPosition(self):
    stop2N = self.previousOrder + 2 * self.atr[-1]
    pyramidHalfN = self.previousOrder - 1/2 * self.atr[-1]
    # in the market, see if we can pyramid our position
    if self.pyramid < 5 and self.dataopen[0] < pyramidHalfN:
        # BUY, BUY, BUY!!! (with default parameters)
        self.log('SELL CREATE PYRAMID, current open price: %.2f' % self.dataopen[0])
        self.log('N value: %.2f' % self.atr[-1])
        self.log('Pyramid: %.2f' % self.pyramid)
        self.log('N buy in price: %.2f' % pyramidHalfN)
        self.pyramid += 1
        # Keep track of the created order to avoid a 2nd order
        self.order = self.sell()
        self.log('New previousOrder, should be same as SELL EXECUTED: %.2f' % self.previousOrder)
        
    # Already in the market ... we might sell
    if self.s1ExitShort[-1] < self.dataopen[0]:
        self.log('BUY CREATE, 10 day high: %.2f' % self.s1ExitShort[-1])
        self.log('exit low: %.2f' % self.dataopen[0])
        # Keep track of the created order to avoid a 2nd order
        self.pyramid = 0
        closeAllPositions(self)
                
    elif self.dataopen[0] > stop2N:
        # Above 2N stop
        stopPrice = stop2N
        self.log('___________BUY CREATE, N stop: %.2f' % stopPrice)
        self.log('Current Price: %.2f' % self.dataopen[0])
        self.log('N value: %.2f' % self.atr[-1])
        # Keep track of the created order to avoid a 2nd order
        self.pyramid = 0
        closeAllPositions(self)

# Create a Stratey
class TestStrategy(bt.Strategy):

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
        self.dataopen = self.datas[0].open

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None
        self.long = None
        self.pyramid = 0
        self.previousOrder = None
        self.totalNumberOfOrders = 0
        self.long = False
        
        # 20 day ATR for N value
        self.atr = bt.indicators.ATR(self.datas[0], period = 20)
        
        # System 1: 20 day entry, 10 day exit
        self.s1EntryLong = bt.indicators.Highest(period = 20)
        self.s1ExitLong = bt.indicators.Lowest(period = 10)

        self.s1EntryShort = bt.indicators.Lowest(period = 20)
        self.s1ExitShort = bt.indicators.Highest(period = 10)
        
        # System 2: 55 day entry, 20 day exit
        self.s2EntryLong = bt.indicators.Highest(period = 55)
        self.s2ExitLong = bt.indicators.Lowest(period = 20)

        self.s2EntryShort = bt.indicators.Lowest(period = 55)
        self.s2ExitShort = bt.indicators.Highest(period = 20)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
                self.previousOrder = order.executed.price
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('|||||||||||||||||OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
        #self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.s1EntryLong[-1] < self.dataopen[0]:

                # BUY, BUY, BUY!!! (with default parameters)
                self.log('BUY CREATE, %.2f' % self.dataopen[0])
                self.log('entry high: %.2f' % self.s1EntryLong[-1])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()
                self.long = True
                #self.long = True

            # we MIGHT SELL if ...
            elif self.s1EntryShort[-1] > self.dataopen[0]:
                self.log('SELL CREATE, %.2f' % self.dataopen[0])
                # self.log('entry high: %.2f' % self.entry[-1])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()
                self.long = False

        else:
            if self.long:
                longPosition(self)
            else:
                shortPosition(self)
           

if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()

    # Add a strategy
    cerebro.addstrategy(TestStrategy)

    # Create a Data Feed
    data = bt.feeds.YahooFinanceCSVData(
        dataname='/Users/Ziyi/Documents/backtrader/data/AAPL.csv', 
        #  fromdate=datetime.datetime(2019, 1, 1), 
        #  todate=datetime.datetime(2019, 12, 31),
        reverse=False)

    # Add the Data Feed to Cerebro
    cerebro.adddata(data)

    # Set our desired cash start
    cerebro.broker.setcash(100000.0)
    
    # Add a FixedSize sizer according to the stake
    cerebro.addsizer(bt.sizers.FixedSize, stake=1)

    # Set the commission - 0.1% ... divide by 100 to remove the %
    cerebro.broker.setcommission(commission=0.001)

    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run()

    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 100000.00
2019-03-04, BUY CREATE, 173.83
2019-03-04, entry high: 173.12
2019-03-05, BUY EXECUTED, Price: 174.08, Cost: 174.08, Comm 0.17
2019-03-08, SELL CREATE, 10 day low: 170.68
2019-03-08, exit low: 168.52
2019-03-11, SELL EXECUTED, Price: 173.63, Cost: 174.08, Comm 0.17
2019-03-11, |||||||||||||||||OPERATION PROFIT, GROSS -0.45, NET -0.80
2019-03-12, BUY CREATE, 178.10
2019-03-12, entry high: 177.01
2019-03-13, BUY EXECUTED, Price: 180.32, Cost: 180.32, Comm 0.18
2019-03-15, BUY CREATE PYRAMID, current open price: 182.89
2019-03-15, N value: 3.23
2019-03-15, Pyramid: 0.00
2019-03-15, N buy in price: 181.93
2019-03-15, New previousOrder, should be same as BUY EXECUTED: 180.32
2019-03-18, BUY EXECUTED, Price: 183.83, Cost: 183.83, Comm 0.18
2019-03-19, BUY CREATE PYRAMID, current open price: 186.36
2019-03-19, N value: 3.21
2019-03-19, Pyramid: 1.00
2019-03-19, N buy in price: 185.44
2019-03-19, New previousOrder, should be same as BUY EXECUTED: 183.83
2019